In [ ]:
# @title save online training dataset (json to pth) (4 class)


SAMPLE_SIZE_PER_CLASS = 58
num_classes = 4
num_word_to_number = {
        "two":2,
        "three":3,
        "four":4,
        "five":5
    }
num_class_to_num_word = {
        2:"two",
        3:"three",
        4:"four",
        5:"five"
    }
def create_online_dataset_texts(
        train_set,
        model,
        processor,
        device="cuda",
        num_class = 4,
        logits_bound=None
):
    print("upper bound", num_class + 1)
    online_dataset=[]
    sample_count_tracking = {}
    # for n in range(2,num_class + 2):
    #     sample_count_tracking[n] = 0

    logits = []
    for i, sample in tqdm(enumerate(train_set)):
        if sample["number"] > num_class + 1:
            continue
        # if sample_count_tracking[sample["number"]] >= SAMPLE_SIZE_PER_CLASS:
        #     continue

        try:
            image = Image.open(requests.get(sample["image_url"], stream=True,timeout=10).raw)
        except Timeout:
            print("timeout")
        except:
            continue


        target_obj_text = sample['target']
        target_obj_with_context_text = sample['target_context'].replace(num_class_to_num_word[sample['number']],"")
        target_obj_aug_text = [f"{num} {sample['target']}" for num in numer_words[:num_classes]]
        target_obj_aug_with_context_text = [sample['target_context'].replace(num_class_to_num_word[sample['number']],num) for num in numer_words[:num_classes]]
        # target_obj_aug_with_context_text,target_obj_text,target_obj_with_context_text,target_obj_aug_text = sentence_augmentation(sample["text"])

        pixel_values=processor(text=None, images=image, return_tensors="pt", padding=True)["pixel_values"] # torch.Size([1, 3, 224, 224])

        image_embeds = get_image_embeds(
            model=model,
            pixel_values=pixel_values,
            device=device,
        )
        target_embeds = text2embedding(target_obj_with_context_text,model,processor,device,True)


        online_dataset.append(
            {
                "gt_count":sample['number'],
                "target_obj_text":target_obj_text,
                "target_obj_aug_text":target_obj_aug_text,
                "target_obj_with_context_text":target_obj_with_context_text,
                "target_obj_aug_with_context_text":target_obj_aug_with_context_text,
                "image_embeds":image_embeds.detach().cpu(),
            }
        )
        # sample_count_tracking[sample["number"]] += 1
        del pixel_values, image_embeds
    # print(sample_count_tracking)
    return online_dataset


def online_data_save_name(model_name,num_class,texts,mode,size,seed,end=""):
    # save_name = f"countbench_{model_name}_{num_class}class_texts{texts}_{SAMPLE_SIZE_PER_CLASS}samples_per_class.pth"
    save_name = f"online_data_{model_name}_{num_class}class_{mode}{size}_texts{texts}_seed{seed}_{end}.pth"
    print(f"Save name: {save_name}")
    return save_name

device="cuda"
use_arabic=False
if use_arabic:
    name_appendix = "_arabic"
    numer_words = ARABIC_NUMBER_WORDS
else:
    name_appendix = ""
    numer_words = NUMBER_WORDS

save_root_folder="./online_data/reviewed_class2345/"
texts=True

for filename in os.listdir(save_root_folder):
    if filename.endswith('.json'): 
        with open(os.path.join(save_root_folder,filename),"r") as f:
            train_set = json.load(f)
    print(len(train_set),filename)
    mode = "train" if "train" in filename else "val"
    seed = int(filename.split("seed")[-1].split(".")[0])

    for idx, model_name in  enumerate(["clip-vit-large-patch14","clip-vit-base-patch32","clip-vit-base-patch16"]): #,"clip-vit-large-patch14"
        model = CLIPModel.from_pretrained(f"openai/{model_name}").to(device)
        model.requires_grad=False
        processor = CLIPProcessor.from_pretrained(f"openai/{model_name}")


        online_dataset = create_online_dataset_texts(
                train_set,
                model,
                processor,
                device=device,
                num_class = 4,
        )
        print(len(online_dataset))
        torch.save(
            online_dataset,
            os.path.join(save_root_folder,online_data_save_name(model_name,4,texts,mode,len(train_set),seed,end=name_appendix))
        )
        # del online_dataset